In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [15]:
game_data = pd.read_csv('preprocess_output.csv')

# LASSO

In [99]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

game_data = game_data.sort_values(by=['TEAM_ID_home', 'GAME_DATE_home'])

game_data.drop(columns=[
    'SEASON_ID_home', 'TEAM_ID_home', 'TEAM_ABBREVIATION_home', 
    'TEAM_NAME_home', 'GAME_ID', 'GAME_DATE_home'
], inplace=True)

# , 'MATCHUP_home', 
#     'WL_home', 'home_away_home', 'SEASON_ID_away', 'TEAM_ID_away', 
#     'TEAM_ABBREVIATION_away', 'TEAM_NAME_away', 'GAME_DATE_away', 
#     'MATCHUP_away', 'WL_away', 'home_away_away'

In [ ]:
data = game_data.dropna()
data = data[9:]

X = data.drop(columns=['WL'])
y = data['WL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LassoCV(cv=5, random_state=42, max_iter=5000))
])

pipeline.fit(X_train, y_train)

In [ ]:
print("Best alpha using built-in LassoCV:", pipeline.named_steps['model'].alpha_)

# Display coefficients
lasso_coefficients = pd.Series(pipeline.named_steps['model'].coef_, index=game_data.drop(columns=['WL']).columns)
print("LASSO Coefficients:\n", lasso_coefficients.sort_values())

# Plot the coefficients
plt.figure(figsize=(12, 6))
lasso_coefficients.plot(kind='bar')
plt.title('Feature Coefficients from LASSO')
plt.show()

In [ ]:
# Select features with non-zero coefficients
selected_features = lasso_coefficients[lasso_coefficients != 0].index
#print("Selected Features:\n", selected_features)

selected_features = selected_features.to_list()
selected_features.append('WL')
selected_features

In [ ]:
lasso = game_data[selected_features]
lasso = lasso[9:]
lasso = lasso.dropna()

X = lasso.drop(columns=['WL'])
y = lasso['WL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
def display_coefficients(pipeline, feature_names):
    coefficients = pipeline.named_steps['model'].coef_[0]
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    })
    return coef_df.sort_values('Coefficient', ascending=False)

display_coefficients(pipeline, X.columns)

# Logistic Regression

I am just going to try a model with this and just see what happens

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
game_data = game_data.sort_values(by=['TEAM_ID_home', 'GAME_DATE_home'])
game_data

In [17]:
game_data.drop(columns=[
    'SEASON_ID_home', 'TEAM_ID_home', 'TEAM_ABBREVIATION_home', 
    'TEAM_NAME_home', 'GAME_ID', 'GAME_DATE_home', 'Unnamed: 0'
], inplace=True)

# , 'MATCHUP_home', 
#     'WL_home', 'home_away_home', 'SEASON_ID_away', 'TEAM_ID_away', 
#     'TEAM_ABBREVIATION_away', 'TEAM_NAME_away', 'GAME_DATE_away', 
#     'MATCHUP_away', 'WL_away', 'home_away_away'

In [ ]:
game_data.columns

In [131]:
data = game_data[[
    'cumulative_FGM_diff', 'cumulative_FGA_diff',
       'cumulative_FG3M_diff', 'cumulative_FG3A_diff', 'cumulative_FTM_diff',
       'cumulative_FTA_diff', 'cumulative_OREB_diff', 'cumulative_DREB_diff',
       'cumulative_REB_diff', 'cumulative_AST_diff', 'cumulative_STL_diff',
       'cumulative_BLK_diff', 'cumulative_TOV_diff', 'cumulative_PTS_diff',
       'cumulative_FG_PCT_diff', 'cumulative_FG3_PCT_diff',
       'cumulative_FT_PCT_diff', 'cumulative_AST_TOV_diff',
       'cumulative_REB_PCT_diff',
    'WL'
]]

In [132]:
data = data.dropna()
data = data[9:]

In [133]:
X = data.drop(columns=['WL'])
y = data['WL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
def display_coefficients(pipeline, feature_names):
    coefficients = pipeline.named_steps['model'].coef_[0]
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    })
    return coef_df.sort_values('Coefficient', ascending=False)

display_coefficients(pipeline, X.columns)

# SVM

In [137]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
svm = SVC(kernel='rbf', C=1.0, gamma='auto')
svm.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_pred = svm.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Random Forest

In [140]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [141]:
# Initialize Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)

# Fit the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

In [ ]:
# Model evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

# Feature importance plot
importances = rf.feature_importances_
feature_names = X.columns

# Sort and plot
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=45)
plt.title("Feature Importances")
plt.show()


# Gradient Boost 

In [144]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model
gb.fit(X_train, y_train)

# Make predictions
y_pred = gb.predict(X_test)

In [ ]:
# Model evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
# Feature importance plot
importances = gb.feature_importances_
feature_names = X.columns

# Sort and plot
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=45)
plt.title("Feature Importances (Gradient Boosting)")
plt.show()
